In [1]:
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

from ratelimit import limits, sleep_and_retry

In [2]:
import Edgar_scrapper

In [135]:
import re

In [3]:
edgar_access = Edgar_scrapper.EdgarAccess()

In [4]:
url = "https://data.sec.gov/submissions/CIK0001018724.json"
json_data = edgar_access.get(url,True)
all_filings = json_data['filings']['recent']

In [7]:
def get_fillings(fillings_cik, doc_type, start=0, count=60):
    fillings_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}\
            &type={}&start={}&count={}&owner=exclude&output=atom'.format(fillings_cik, doc_type, start, count)
    fillings_html = edgar_access.get(fillings_url)
    fillings_soup = BeautifulSoup(fillings_html, features="html.parser")

    fillings_list = [
        (link.find('filing-href').getText(),
         link.find('filing-type').getText(),
         link.find('filing-date').getText())
        for link in fillings_soup.find_all('entry')]

    return fillings_list

In [105]:
ticker_ciks = pd.read_csv('tickers.csv')

In [187]:
sample_ticker = pd.DataFrame(data={'ticker' : ['AMC'],'cik':[1411579]})

In [189]:
sec_fillings_indx = {}

In [190]:
for ticker, cik in sample_ticker[:1].values:
    sec_fillings_indx[ticker] = get_fillings(cik, '10-K')

In [191]:
sec_fillings = {}
for ticker, data in sec_fillings_indx.items():
    sec_fillings[ticker] = {}
    for filing_href, filing_type, filing_date in tqdm(data, desc='Downloading {} Fillings'.format(ticker), unit='filling'):
        if (filing_type == '10-K'):
            #file_url = filing_href.replace('-index.htm', '.txt').replace('.txtl', '.txt')
            sec_fillings[ticker][filing_date] = edgar_access.get(filing_href)

In [192]:
annual_reports_df = pd.DataFrame(columns=['ticker','date','10k-link'])

In [193]:
for ticker in sec_fillings.keys():
    for date in sec_fillings[ticker].keys():
        fillings_links_soup = BeautifulSoup(sec_fillings[ticker][date], features="html.parser")
        for tr_tag in fillings_links_soup.find_all('tr'):
            for td_tag in tr_tag.find_all('td'):
                if(td_tag.text == '10-K'):
                    href = tr_tag.find('a')["href"]
                    pattern = r"^/"
                    complete_href = re.sub(pattern,"https://www.sec.gov/",href)
                    annual_reports_df = annual_reports_df.append({'ticker':ticker,'date':date,'10k-link':complete_href},ignore_index=True)

In [195]:
annual_reports_df.values

array([['AMC', '2021-03-12',
        'https://www.sec.gov/ix?doc=/Archives/edgar/data/1411579/000141157921000006/amc-20201231x10k.htm'],
       ['AMC', '2021-03-12',
        'https://www.sec.gov/ix?doc=/Archives/edgar/data/1411579/000141157921000006/amc-20201231x10k.htm'],
       ['AMC', '2020-02-28',
        'https://www.sec.gov/ix?doc=/Archives/edgar/data/1411579/000141157920000027/amc-20191231x10k.htm'],
       ['AMC', '2020-02-28',
        'https://www.sec.gov/ix?doc=/Archives/edgar/data/1411579/000141157920000027/amc-20191231x10k.htm'],
       ['AMC', '2018-03-01',
        'https://www.sec.gov/Archives/edgar/data/1411579/000141157918000014/c579-20171231x10k.htm'],
       ['AMC', '2018-03-01',
        'https://www.sec.gov/Archives/edgar/data/1411579/000141157918000014/c579-20171231x10k.htm'],
       ['AMC', '2017-03-10',
        'https://www.sec.gov/Archives/edgar/data/1411579/000141157917000021/amc-20161231x10k.htm'],
       ['AMC', '2017-03-10',
        'https://www.sec.gov/Archi

In [95]:
for tr_tag in fillings_links_soup.find_all('tr'):
    for td_tag in tr_tag.find_all('td'):
        if(td_tag.text == '10-K'):
          print(tr_tag.find('a')["href"].replace("/ix?doc=","https://www.sec.gov/"))

https://www.sec.gov//Archives/edgar/data/14272/000001427221000066/bmy-20201231.htm
https://www.sec.gov//Archives/edgar/data/14272/000001427221000066/bmy-20201231.htm
